## Packages
[microdatasus](https://github.com/rfsaldanha/microdatasus)

[read.dbc](https://github.com/danicat/read.dbc)

## SINASC data dictionary

https://github.com/rfsaldanha/microdatasus/wiki/Conven%C3%A7%C3%B5es-SINASC

 ## Code

The *microdatasus* package can't find SINASC (Live Birth Information System) data from 2017 ownwards. The package is trying to download data from "ftp://ftp.datasus.gov.br/dissemin/publicos/SINASC/NOV", but "preliminary" data is apparently stored at "ftp://ftp.datasus.gov.br/dissemin/publicos/SINASC/PRELIM" - PRELIM for 2021, PRELIM20 for 2020, and so on.

An alternative is to use the package *read.dbc* to directly read data from the DATASUS server. "DBC" is the format used by the brazilian government to store these data.

## Data to fetch
- Year: 2017-2021
- Location: "AC", "AL", "AP", "AM", "BA", "CE", "DF", "ES", "GO", "MA", "MT", "MS", "MG", "PA", "PB", "PR", "PE", "PI", "RJ", "RN", "RS", "RO", "RR", "SC", "SP", "SE", "TO"

Reference link: http://tabnet.datasus.gov.br/cgi/tabcgi.exe?sinasc/cnv/nvuf.def

State abbreviation: https://sco.wikipedia.org/wiki/States_o_Brazil

- Notes:
    - datasets before 2021 are residing in both **DRES** and **PRELIM** folder, and datasets in 2021 only reside in **PRELIM** folder.
    - The datasets in **DRES** and **PRELIM** are not exactly the same. After exploration, we found that datasets in **DRES** contains less NA compared with **PRELIM**. 


In [ ]:
install.packages("read.dbc")
library(read.dbc)
library(dplyr)

Installing package into ‘/home/jupyter/.R/library’
(as ‘lib’ is unspecified)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [ ]:
locations = c("AC", "AL", "AP", "AM", "BA", "CE", "DF", "ES", "GO", "MA", "MT", "MS", "MG", "PA", "PB", 
              "PR", "PE", "PI", "RJ", "RN", "RS", "RO", "RR", "SC", "SP", "SE", "TO")

years = c(2015, 2016, 2017, 2018, 2019, 2020, 2021)

In [ ]:
shell_call <- function(command, ...) {
  result <- system(command, intern = TRUE, ...)
  cat(paste0(result, collapse = "\n"))
}

columns_to_keep = c("CODESTAB", "CODMUNNASC", "LOCNASC", "IDADEMAE", "ESTCIVMAE", "ESCMAE", "CODOCUPMAE", "QTDFILVIVO",
                    "QTDFILMORT", "CODMUNRES", "GESTACAO", "GRAVIDEZ", "PARTO", "CONSULTAS", "DTNASC", "HORANASC", "SEXO",
                    "APGAR1", "APGAR5", "RACACOR", "PESO", "IDANOMAL", "DTCADASTRO", "CODANOMAL", "ESCMAE2010")

columns_new_name = c("health_establishment", "municipality_code", "place_of_birth", "mother_age", "mother_marital",
                      "mother_education", "mother_occupation", "living_children", "dead_children", "mother_residence",
                      "pregnancy_weeks", "pregnancy_type", "delivery_type", "prenatal_consultations", "birth_date",
                      "birth_time", "sex", "apgar1", "apgar5", "race_color", "birth_weight", "congenital_anomaly",
                      "registration_date", "congenital_anomaly_code", "mother_schooling")

get_data <- function(state, year) {
  ftp_folder <- paste0('ftp://ftp.datasus.gov.br/dissemin/publicos/SINASC/NOV/DNRES/')
  if(year == 2021) {
    # a more primitive version 
    ftp_folder <- "ftp://ftp.datasus.gov.br/dissemin/publicos/SINASC/PRELIM/DNRES/"
  }
  dbc_file <- paste0("DN", state, year, ".DBC")
  url <- paste0(ftp_folder, dbc_file)
  shell_call(paste("wget", url))
  dnsp <- read.dbc(dbc_file)
  file.remove(dbc_file)
  df <- select(dnsp, all_of(columns_to_keep))
  colnames(df) = columns_new_name
  # df <- df %>% filter(!is.na(congenital_anomaly))
  df["Area"] = state
  df["Year"] = year
  print(paste0("Number of Living Birth record: ", nrow(df)))
  print(paste0("Number of Variables: ", ncol(df)))
  filename = paste0("LivingBirth", state, year, ".csv")
  write.csv(df, paste0("data_nomissing/",filename))
}

In [ ]:
for(i in 1:length(years)){
  for(j in 1:length(locations)){
    print(paste0("Year:", years[i], " ", "Location:", locations[j]))
    get_data(locations[j], years[i])
  }
}

[1] "Year:2015 Location:AC"
[1] "Number of Living Birth record: 16980"
[1] "Number of Variables: 27"
[1] "Year:2015 Location:AL"
[1] "Number of Living Birth record: 52257"
[1] "Number of Variables: 27"
[1] "Year:2015 Location:AP"
[1] "Number of Living Birth record: 15750"
[1] "Number of Variables: 27"
[1] "Year:2015 Location:AM"
[1] "Number of Living Birth record: 80097"
[1] "Number of Variables: 27"
[1] "Year:2015 Location:BA"
[1] "Number of Living Birth record: 206655"
[1] "Number of Variables: 27"
[1] "Year:2015 Location:CE"
[1] "Number of Living Birth record: 132516"
[1] "Number of Variables: 27"
[1] "Year:2015 Location:DF"
[1] "Number of Living Birth record: 46122"
[1] "Number of Variables: 27"
[1] "Year:2015 Location:ES"
[1] "Number of Living Birth record: 56941"
[1] "Number of Variables: 27"
[1] "Year:2015 Location:GO"
[1] "Number of Living Birth record: 100672"
[1] "Number of Variables: 27"
[1] "Year:2015 Location:MA"
[1] "Number of Living Birth record: 117564"
[1] "Number of V

## Variables:
- CODESTAB: Health establishment code 
- ORIGEM: unknown
- CODMUNNASC: Code of the municipality of occurrence
- LOCNASC: Place of birth occurrence
- IDADEMAE:Mother's age in years
- ESTCIVMAE: Mother's Marital status
- ESCMAE: Mother's Education, years of study completed
- CODOCUPMAE: Mother's Occupation, according to the Brazilian Classification of Occupations
- QTDFILVIVO: Number of living children
- QTDFILMORT: Number of dead children
- CODMUNRES: Municipality of residence of the mother
- GESRACAO: Weeks of pregnancy
- GRAVIDEZ: Type of pregnancy
- PARTO: Type of delivery
- CONSULTAS: Number of prenatal consultations
- DTNASC: Date of birth
- HORANASC: Time of birth
- SEXO: Sex
- APGAR1: APGAR in the first minute
- APGAR5: APGAR in the fifth minute
- RACACOR:  Race/color
- PESO: Birth weight, in grams
- IDANOMAL: Congenital anomaly
- DTCADASTRO: Date of registration in the system
- CODANOMAL: Code of congenital malformation or chromosomal anomaly, according to ICD-10
- NUMEROLOTE: unknown
- VERSAOSIST: unknown
- DTRECEBIM: Date of receipt at the central level, date of the last registration update
- DIFDATA: unknown
- DTRECORIGA: unknown
- NATURALMAE: unknown 
- CODMUNNATU: unknown
- CODUFNATU: unknown
- ESCMAE2010: Mother's schooling classified from 2010
- SERIESCMAE: unknown
- DTNASCMAE: unknown
- RACACORMAE: unknown
- QTDGESTANT: unknown
- QTDPARTNOR: unknown
- QTDPARTCES: unknown
- IDADEPAI: unknown
- DTULTMENST: unknown
- SEMAGESTAC: unknown
- TPMETESTIM: unknown
- CONSPRENAT: unknown
- MESPRENAT: unknown
- TPAPRESENT: unknown
- STTRABPART: unknown
- STCESPARTO: unknown
- TPNASCASSI: unknown
- TPFUNCRESP: unknown
- TPDOCRESP: unknown
- DTDECLARAC: unknown
- ESCMAEAGR1: unknown
- STDNEPIDEM: unknown
- STDNNOVA: unknown
- CODPAISRES: unknown
- TPROBSON: unknown
- PARIDADE: unknown
- KOTELCHUCK: unknown
- CONTADOR: unknown

Reference link: https://github.com/rfsaldanha/microdatasus/wiki/Convenções-SINASC#numerodn


### Data Cleaning Plan:
- drop all unknown variables
- discard useless date such as registration date and data receipt date
- May also consider delete establishment code or trivial location information.
- current variables we have: **health_establishment, municipality_code, place_of_birth mother_age, mother_marital, mother_education, mother_occupation, living_children, dead_children, mother_residence, pregnancy_weeks, pregnancy_type, delivery_type, prenatal_consultations, birth_date,birth_time, sex,  apgar1, apgar5, race_color,birth_weight, congenital_anomaly, registration_date, congenital_anomaly_code, mother_schooling.**
- missing value: 
  - drop rows that have NA in cogenital malformation
  - other columns also have a few NAs, may do median/mean value imputation



### Modeling Plan:
- variables going to use: 
- prediction target: Classification problem, whether a child will have congenital malformation or not.
- evaluation metrics:
specificity, sensitivity, AUC-ROC.
- programming language: after uploading all the dataset to Google Cloud, I would prefer to use Python for data visualization and model building.

# Aggregate the datasets in the same year for data visualization

In [ ]:
file_list = list.files(path="datasets", pattern=NULL, all.files=FALSE, full.names=TRUE)

In [ ]:
file_list

[1] "datasets/LivingBirthAC2015.csv" "datasets/LivingBirthAC2016.csv"
  [3] "datasets/LivingBirthAC2017.csv" "datasets/LivingBirthAC2018.csv"
  [5] "datasets/LivingBirthAC2019.csv" "datasets/LivingBirthAC2020.csv"
  [7] "datasets/LivingBirthAC2021.csv" "datasets/LivingBirthAL2015.csv"
  [9] "datasets/LivingBirthAL2016.csv" "datasets/LivingBirthAL2017.csv"
 [11] "datasets/LivingBirthAL2018.csv" "datasets/LivingBirthAL2019.csv"
 [13] "datasets/LivingBirthAL2020.csv" "datasets/LivingBirthAL2021.csv"
 [15] "datasets/LivingBirthAM2015.csv" "datasets/LivingBirthAM2016.csv"
 [17] "datasets/LivingBirthAM2017.csv" "datasets/LivingBirthAM2018.csv"
 [19] "datasets/LivingBirthAM2019.csv" "datasets/LivingBirthAM2020.csv"
 [21] "datasets/LivingBirthAM2021.csv" "datasets/LivingBirthAP2015.csv"
 [23] "datasets/LivingBirthAP2016.csv" "datasets/LivingBirthAP2017.csv"
 [25] "datasets/LivingBirthAP2018.csv" "datasets/LivingBirthAP2019.csv"
 [27] "datasets/LivingBirthAP2020.csv" "datasets/LivingBirthAP2021.csv"
 [29] "datasets/LivingBirthBA2015.csv" "datasets/LivingBirthBA2016.csv"
 [31] "datasets/LivingBirthBA2017.csv" "datasets/LivingBirthBA2018.csv"
 [33] "datasets/LivingBirthBA2019.csv" "datasets/LivingBirthBA2020.csv"
 [35] "datasets/LivingBirthBA2021.csv" "datasets/LivingBirthCE2015.csv"
 [37] "datasets/LivingBirthCE2016.csv" "datasets/LivingBirthCE2017.csv"
 [39] "datasets/LivingBirthCE2018.csv" "datasets/LivingBirthCE2019.csv"
 [41] "datasets/LivingBirthCE2020.csv" "datasets/LivingBirthCE2021.csv"
 [43] "datasets/LivingBirthDF2015.csv" "datasets/LivingBirthDF2016.csv"
 [45] "datasets/LivingBirthDF2017.csv" "datasets/LivingBirthDF2018.csv"
 [47] "datasets/LivingBirthDF2019.csv" "datasets/LivingBirthDF2020.csv"
 [49] "datasets/LivingBirthDF2021.csv" "datasets/LivingBirthES2015.csv"
 [51] "datasets/LivingBirthES2016.csv" "datasets/LivingBirthES2017.csv"
 [53] "datasets/LivingBirthES2018.csv" "datasets/LivingBirthES2019.csv"
 [55] "datasets/LivingBirthES2020.csv" "datasets/LivingBirthES2021.csv"
 [57] "datasets/LivingBirthGO2015.csv" "datasets/LivingBirthGO2016.csv"
 [59] "datasets/LivingBirthGO2017.csv" "datasets/LivingBirthGO2018.csv"
 [61] "datasets/LivingBirthGO2019.csv" "datasets/LivingBirthGO2020.csv"
 [63] "datasets/LivingBirthGO2021.csv" "datasets/LivingBirthMA2015.csv"
 [65] "datasets/LivingBirthMA2016.csv" "datasets/LivingBirthMA2017.csv"
 [67] "datasets/LivingBirthMA2018.csv" "datasets/LivingBirthMA2019.csv"
 [69] "datasets/LivingBirthMA2020.csv" "datasets/LivingBirthMA2021.csv"
 [71] "datasets/LivingBirthMG2015.csv" "datasets/LivingBirthMG2016.csv"
 [73] "datasets/LivingBirthMG2017.csv" "datasets/LivingBirthMG2018.csv"
 [75] "datasets/LivingBirthMG2019.csv" "datasets/LivingBirthMG2020.csv"
 [77] "datasets/LivingBirthMG2021.csv" "datasets/LivingBirthMS2015.csv"
 [79] "datasets/LivingBirthMS2016.csv" "datasets/LivingBirthMS2017.csv"
 [81] "datasets/LivingBirthMS2018.csv" "datasets/LivingBirthMS2019.csv"
 [83] "datasets/LivingBirthMS2020.csv" "datasets/LivingBirthMS2021.csv"
 [85] "datasets/LivingBirthMT2015.csv" "datasets/LivingBirthMT2016.csv"
 [87] "datasets/LivingBirthMT2017.csv" "datasets/LivingBirthMT2018.csv"
 [89] "datasets/LivingBirthMT2019.csv" "datasets/LivingBirthMT2020.csv"
 [91] "datasets/LivingBirthMT2021.csv" "datasets/LivingBirthPA2015.csv"
 [93] "datasets/LivingBirthPA2016.csv" "datasets/LivingBirthPA2017.csv"
 [95] "datasets/LivingBirthPA2018.csv" "datasets/LivingBirthPA2019.csv"
 [97] "datasets/LivingBirthPA2020.csv" "datasets/LivingBirthPA2021.csv"
 [99] "datasets/LivingBirthPB2015.csv" "datasets/LivingBirthPB2016.csv"
[101] "datasets/LivingBirthPB2017.csv" "datasets/LivingBirthPB2018.csv"
[103] "datasets/LivingBirthPB2019.csv" "datasets/LivingBirthPB2020.csv"
[105] "datasets/LivingBirthPB2021.csv" "datasets/LivingBirthPE2015.csv"
[107] "datasets/LivingBirthPE2016.csv" "datasets/LivingBirthPE2017.csv"
[109] "datasets/LivingBirthPE2018.csv" "datasets/LivingBirthPE2019.csv"
[111] "datasets/LivingBirthPE2020.csv" "da

In [ ]:
conbine_file = function(year, file_list){
  df = data.frame(matrix(ncol = length(columns_new_name), nrow = 0))
  colnames(df) = columns_new_name
  for(i in 1:length(file_list)){
    if(grepl(toString(year), file_list[i], fixed=TRUE)){
      new_df = read.csv(file = file_list[i])
      df = rbind(df, new_df)
    }
  }
  new_file_name = paste0("LB", year, ".csv")
  # can consider write the dataframe out to disk
  write.csv(df, paste0("combined_data/", new_file_name))
  return(df)
}

In [ ]:
df2020 = conbine_file(2021, file_list)

In [ ]:
dim(df2020)

[1] 2652245      28

In [ ]:
head(df2020)

,X,health_establishment,municipality_code,place_of_birth,mother_age,mother_marital,mother_education,mother_occupation,living_children,dead_children,⋯,apgar1,apgar5,race_color,birth_weight,congenital_anomaly,registration_date,congenital_anomaly_code,mother_schooling,Area,Year
,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<chr>,<int>,<chr>,<int>
1,1,2515768,110010,1,18,1,4,NA,0,1,⋯,7,9,4,2860,2,22042021,NA,3,AC,2021
2,2,2516276,110015,1,28,2,5,NA,0,0,⋯,5,8,1,700,2,18022021,NA,5,AC,2021
3,3,2496879,110015,1,32,5,4,999992,1,0,⋯,8,9,4,3250,2,17062021,NA,3,AC,2021
4,4,5618347,110020,1,25,NA,3,999992,4,1,⋯,9,10,4,3720,2,1062021,NA,2,AC,2021
5,5,5618347,110020,1,32,2,5,999992,1,2,⋯,9,10,4,3725,2,26072021,NA,4,AC,2021
6,6,5701929,120001,1,25,2,4,NA,3,0,⋯,7,8,2,3500,2,5022021,NA,2,AC,2021


In [ ]:
df2019 = conbine_file(2019, file_list)
df2018 = conbine_file(2018, file_list)
df2017 = conbine_file(2017, file_list)
df2016 = conbine_file(2016, file_list)
df2015 = conbine_file(2015, file_list)

In [ ]:
df2015 = conbine_file(2015, file_list)